In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
from scipy import stats
from scipy.stats import zscore
from scipy.interpolate import interp1d
from glob import glob
import csv
import seaborn as sns
import string
from string import digits
import colorcet as cc
import ast
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

sns.set(font_scale=1)


In [8]:
# General notes:
# subjects don't have session 1&2 data: e152012, e152048
# subjects have missing segmentation data: e152015, e152053 (both didn't complete coarse)
# subjects with split segmentation files that need a workaround: e152019, e152026, e152027, e152028, e152029,e152037, e152043


In [9]:
# There are differences in the column names psychopy output, so it's important to load participants separately by condition
# note that cf doesn't necessarily mean segmentation data for the participant is coarse-fine, for example: subject 26's recognition/recall data is in cf format, but they did the fine then coarse segmentation. This is because psychopy crashed while doing segmentation, so they had to restart the experiment in a different order.
cf_subjects = ['e152006', 'e152007', 'e152009', 'e152010', 'e152014', 'e152016',
               'e152018', 'e152020v2', 'e152022', 'e152024', 'e152026', 'e152027', 'e152028', 'e152029', 'e152030', 'e152032', 'e152034', 'e152036', 'e152038', 'e152042', 'e152045', 'e152047', 'e152050', 'e152052']
fc_subjects = ['e152005', 'e152008', 'e152011', 'e152013', 'e152015', 'e152017', 'e152019',
               'e152021', 'e152023', 'e152025', 'e152031', 'e152033', 'e152035', 'e152037', 'e152039', 'e152040', 'e152041', 'e152043', 'e152044', 'e152046', 'e152049', 'e152051', 'e152053']

# sum to 47 corresponding to 47 fMRI participants, good!
len(cf_subjects), len(fc_subjects)


(24, 23)

In [10]:
def check_in_cf_fc(s2file):
    sub = os.path.basename(s2file).split('_')[0]
    if sub in cf_subjects + fc_subjects:
        return True
    else:
        return False


# Load recall/recognition output files from PsychoPy:
s2files = glob(
    r'C:\Users\nguye\Box\DCL_ARCHIVE\Documents\Events\exp152_fMRIneuralmechanisms\exp152_Session_2\data\e*.csv')

# there are duplicates of the same participant, and we want to use the most updated version only, which is in the cf/fc lists.
s2files = [s2file for s2file in s2files if check_in_cf_fc(s2file)]


# Recall

In [5]:
# TODO: overall, make sure to select the right columns, and make sure the difference between FC and CF (that has different column names) is accounted for.
# TODO: pay attention to idyo subjects raw data files and how they're processed.
# TODO: there're lots of exceptions raised here, come back later
'''
FREE RECALL
'''
# subject 28 miss one recall data for video 2.4.1
# Recall rows can be indexed by actor face shown:
faces = ['images/1.2.3.face.png', 'images/6.3.9.face.png',
         'images/3.1.3.face.png', 'images/2.4.1.face.png']
recalldf = pd.DataFrame()

for s2file in s2files:
    try:
        # s2file = r'C:\Users\nguye\Box\DCL_ARCHIVE\Documents\Events\exp152_fMRIneuralmechanisms\exp152_Session_2\data\e152028_exp152_session_2_2021_Dec_17_0751.csv'
        sub = os.path.basename(s2file).split('_')[0]

        df = pd.read_csv(s2file)
        for i, face in enumerate(faces):
            # recall is in textbox.text column
            # print(df[df.face==face]['textbox.text'].values[0])
            recall = df[df.face == face]['textbox.text'].values[0]
            filtered_sentence = []
            # Remove digits
            remove_digits = str.maketrans('', '', digits)
            try:
                res = recall.translate(remove_digits)
            except AttributeError:
                print("Attribute Error: ", recall, s2file)
            # Make lower case
            sl = res.lower()
            # Remove punctuation
            table = str.maketrans(
                {key: None for key in string.punctuation})
            slnp = sl.translate(table)
            split_str = slnp.split()
            filtered_sentence = split_str
            # Remove stopwords
            #filtered_sentence = [w for w in split_str if not w in s_words]

            # print(filtered_sentence)
            tdf = pd.DataFrame({'sub': [sub], 'movie': [face.split('/')[1][:-9]], 'recall_raw': [
                                recall], 'recall_filtered': [filtered_sentence], 'filtered_length': len(filtered_sentence)})
            recalldf = pd.concat([recalldf, tdf], ignore_index=True)
    except Exception as e:
        print(e)
        print(s2file)


Attribute Error:  nan C:\Users\nguye\Box\DCL_ARCHIVE\Documents\Events\exp152_fMRIneuralmechanisms\exp152_Session_2\data\e152028_exp152_session_2_2021_Dec_17_0751.csv


In [6]:
recalldf['filtered_length_scaled'] = stats.zscore(recalldf['filtered_length'])
recalldf


,sub,movie,recall_raw,recall_filtered,filtered_length,filtered_length_scaled
0,e152005,1.2.3,This woman walked into a room with a table on...,"[this, woman, walked, into, a, room, with, a, ...",193,0.606046
1,e152005,6.3.9,This actor was folding laundry in the video. H...,"[this, actor, was, folding, laundry, in, the, ...",98,-0.330204
2,e152005,3.1.3,This actor was preparing breakfast in the vide...,"[this, actor, was, preparing, breakfast, in, t...",140,0.083717
3,e152005,2.4.1,This actor was in the bathroom in the video. H...,"[this, actor, was, in, the, bathroom, in, the,...",63,-0.675138
4,e152006,1.2.3,This lady came in from the left side of the r...,"[this, lady, came, in, from, the, left, side, ...",287,1.532441
...,...,...,...,...,...,...
183,e152052,2.4.1,\nThis guy walked into his bathroom and I beli...,"[this, guy, walked, into, his, bathroom, and, ...",217,0.842573
184,e152053,1.2.3,A woman enters a room from the left side of t...,"[a, woman, enters, a, room, from, the, left, s...",159,0.270967
185,e152053,6.3.9,A man entered the room with a basket of laundr...,"[a, man, entered, the, room, with, a, basket, ...",78,-0.527309
186,e152053,3.1.3,A man walks into what appears to be an apartme...,"[a, man, walks, into, what, appears, to, be, a...",182,0.497638


# Recognition

In [7]:
'''
RECOGNITION
'''
# NOTE: There's a bug in the experiment script for auto-calculating accuracy on run 3 (3.1.3) for subjects e152003 and e152004,
# so accuracy should be re-calculated based on the correct response and subject response columns
# Also, for these two participants, the successive runs are in different columns, requiring work-arounds.

recogdf = pd.DataFrame()
for s2file in s2files:
    sub = os.path.basename(s2file).split('_')[0]
    try:
        # s2file='/Users/bezdek/Box/DCL_ARCHIVE/Documents/Events/exp152_fMRIneuralmechanisms/exp152_Session_2/data/e152011_exp152_session_2_2021_Aug_30_1305.csv'
        if sub in ['e152003', 'e152004']:
            df = pd.read_csv(s2file)
            df['key_resp.keys'] = df['key_resp.keys'].combine_first(
                df['key_resp_7.keys'])
            df['key_resp.rt'] = df['key_resp.rt'].combine_first(
                df['key_resp_7.rt'])
            df['key_resp.keys'] = df['key_resp.keys'].combine_first(
                df['key_resp_8.keys'])
            df['key_resp.rt'] = df['key_resp.rt'].combine_first(
                df['key_resp_8.rt'])
            df['key_resp.keys'] = df['key_resp.keys'].combine_first(
                df['key_resp_9.keys'])
            df['key_resp.rt'] = df['key_resp.rt'].combine_first(
                df['key_resp_9.rt'])
            df['recog_resp.corr'] = np.where(((df['key_resp.keys'] == 'right') & (df['corrAns'] == 'right')) | (
                (df['key_resp.keys'] == 'left') & (df['corrAns'] == 'left')), 1, 0)
            df = df.rename(
                columns={'key_resp.keys': 'recog_resp.keys', 'key_resp.rt': 'recog_resp.rt'})
            df = df[['movie', 'task', 'image', 'corrAns', 'trial_type',
                     'recog_resp.keys', 'recog_resp.corr', 'recog_resp.rt']]
        elif sub in fc_subjects:
            df = pd.read_csv(s2file)
            # select columns and rows:
            df = df[['movie', 'task', 'image', 'corrAns', 'trial_type',
                     'recog_resp.keys', 'recog_resp.corr', 'recog_resp.rt']]
        elif sub in cf_subjects:
            df = pd.read_csv(s2file)
            # select columns and rows:
            df = df[['movie', 'task', 'image', 'corrAns', 'trial_type',
                     'key_resp.keys', 'key_resp.corr', 'key_resp.rt']]
            df = df.rename(columns={'key_resp.keys': 'recog_resp.keys',
                           'key_resp.corr': 'recog_resp.corr', 'key_resp.rt': 'recog_resp.rt'})

        df = df[(df.task == 'recog') & (df.movie != 'legos2')]
        df = df.groupby(['movie', 'trial_type']).mean(numeric_only=True)
        df['sub'] = sub
        df['movie'] = [x[0] for x in df.index]
        df['trial_type'] = [x[1] for x in df.index]
        recogdf = pd.concat([recogdf, df], ignore_index=True)
    except Exception as e:
        print(e, sub)
        print(s2file)


In [8]:
recogdf


,recog_resp.corr,recog_resp.rt,sub,movie,trial_type
0,0.550000,1.254202,e152005,1.2.3,lure
1,0.894737,1.146668,e152005,1.2.3,old
2,0.650000,1.619908,e152005,2.4.1,lure
3,0.857143,2.063136,e152005,2.4.1,old
4,0.700000,2.979255,e152005,3.1.3,lure
...,...,...,...,...,...
371,0.952381,1.657716,e152053,2.4.1,old
372,0.750000,4.222865,e152053,3.1.3,lure
373,0.947368,2.427225,e152053,3.1.3,old
374,0.450000,2.537419,e152053,6.3.9,lure


In [9]:
# pivot to wide format
recogdf_wide = recogdf.pivot_table(
    index=['sub', 'movie'], columns='trial_type', values='recog_resp.corr').reset_index()
recogdf_wide


trial_type,sub,movie,lure,old
0,e152005,1.2.3,0.55,0.894737
1,e152005,2.4.1,0.65,0.857143
2,e152005,3.1.3,0.70,1.000000
3,e152005,6.3.9,0.50,0.904762
4,e152006,1.2.3,0.90,0.947368
...,...,...,...,...
183,e152052,6.3.9,0.80,0.666667
184,e152053,1.2.3,0.95,1.000000
185,e152053,2.4.1,0.40,0.952381
186,e152053,3.1.3,0.75,0.947368


In [13]:
# compute sensitivity (d') and bias/criterion
recogdf_wide['dprime'] = 0
recogdf_wide['bias'] = 0
for sub in recogdf_wide['sub'].unique():
    for movie in recogdf_wide.movie.unique():
        tempdf = recogdf_wide[(recogdf_wide['sub'] == sub)
                              & (recogdf_wide['movie'] == movie)]
        # add correction for 100 percent hits / correct rejections
        hit = tempdf.old.item()
        # Correct for perfect
        if hit == 1:
            hit = .975
        if hit == 0:
            hit = .025
        miss = 1 - hit
        cr = tempdf.lure.item()
        if cr == 1:
            cr = .975
        if cr == 0:
            cr = .025
        fa = 1 - cr

        dprime = stats.norm.ppf(hit) - stats.norm.ppf(fa)
        c = -(stats.norm.ppf(hit) + stats.norm.ppf(fa))/2.0
        recogdf_wide.loc[(recogdf_wide['sub'] == sub) & (
            recogdf_wide['movie'] == movie), 'dprime'] = dprime.item()
        recogdf_wide.loc[(recogdf_wide['sub'] == sub) & (
            recogdf_wide['movie'] == movie), 'bias'] = c.item()


In [14]:
recogdf_wide


trial_type,sub,movie,lure,old,dprime,bias
0,e152005,1.2.3,0.55,0.894737,1.377781,-0.563229
1,e152005,2.4.1,0.65,0.857143,1.452891,-0.341125
2,e152005,3.1.3,0.70,1.000000,2.484364,-0.717782
3,e152005,6.3.9,0.50,0.904762,1.309172,-0.654586
4,e152006,1.2.3,0.90,0.947368,2.901408,-0.169152
...,...,...,...,...,...,...
183,e152052,6.3.9,0.80,0.666667,1.272349,0.205447
184,e152053,1.2.3,0.95,1.000000,3.604818,-0.157555
185,e152053,2.4.1,0.40,0.952381,1.415044,-0.960869
186,e152053,3.1.3,0.75,0.947368,2.294346,-0.472683


# Recall+Recognition

In [15]:
memory_df = pd.merge(recalldf, recogdf_wide, on=['sub', 'movie'])
memory_df

,sub,movie,recall_raw,recall_filtered,filtered_length,filtered_length_scaled,lure,old,dprime,bias
0,e152005,1.2.3,This woman walked into a room with a table on...,"[this, woman, walked, into, a, room, with, a, ...",193,0.606046,0.55,0.894737,1.377781,-0.563229
1,e152005,6.3.9,This actor was folding laundry in the video. H...,"[this, actor, was, folding, laundry, in, the, ...",98,-0.330204,0.50,0.904762,1.309172,-0.654586
2,e152005,3.1.3,This actor was preparing breakfast in the vide...,"[this, actor, was, preparing, breakfast, in, t...",140,0.083717,0.70,1.000000,2.484364,-0.717782
3,e152005,2.4.1,This actor was in the bathroom in the video. H...,"[this, actor, was, in, the, bathroom, in, the,...",63,-0.675138,0.65,0.857143,1.452891,-0.341125
4,e152006,1.2.3,This lady came in from the left side of the r...,"[this, lady, came, in, from, the, left, side, ...",287,1.532441,0.90,0.947368,2.901408,-0.169152
...,...,...,...,...,...,...,...,...,...,...
183,e152052,2.4.1,\nThis guy walked into his bathroom and I beli...,"[this, guy, walked, into, his, bathroom, and, ...",217,0.842573,0.65,0.857143,1.452891,-0.341125
184,e152053,1.2.3,A woman enters a room from the left side of t...,"[a, woman, enters, a, room, from, the, left, s...",159,0.270967,0.95,1.000000,3.604818,-0.157555
185,e152053,6.3.9,A man entered the room with a basket of laundr...,"[a, man, entered, the, room, with, a, basket, ...",78,-0.527309,0.45,0.523810,-0.065944,-0.092689
186,e152053,3.1.3,A man walks into what appears to be an apartme...,"[a, man, walks, into, what, appears, to, be, a...",182,0.497638,0.75,0.947368,2.294346,-0.472683


In [30]:
px.scatter(memory_df.dropna(axis=0), x='filtered_length_scaled', 
           marginal_x='histogram', marginal_y='histogram',
           y='dprime', 
        #    color='sub',
           hover_data=['sub', 'bias', 'movie'], trendline='ols')

In [28]:
px.scatter(memory_df.dropna(axis=0), x='bias', 
           marginal_x='histogram',
           marginal_y='histogram',
           y='dprime', 
       #     color='sub',
           hover_data=['sub', 'bias'], trendline='ols')

In [75]:
memory_df[(memory_df['sub'] == 'e152022') & (memory_df['movie'] == '3.1.3')].recall_raw

66    This actor came into the kitchen, retrieved a ...
Name: recall_raw, dtype: object

In [76]:
memory_df.to_csv('memory_df.csv')

# Segmentation

In [35]:
# attempt to re-run Matt's code, but there are many errors
segdf=pd.DataFrame()
for s2file in s2files:
    try:
        #s2file='/Users/bezdek/Box/DCL_ARCHIVE/Documents/Events/exp152_fMRIneuralmechanisms/exp152_Session_2/data/e152011_exp152_session_2_2021_Aug_30_1305.csv'
        sub = os.path.basename(s2file).split('_')[0]
        if sub in ['e152003','e152004']:
            pass      
        elif sub in fc_subjects:
            df=pd.read_csv(s2file)
            # select columns and rows:
            df=df[['movie','task','segment_coarse.rt','segment_fine.rt']]
            df['order']='fine_coarse'
            df = df[(df.task == 'segment') & (df.movie != 'legos2')]
            df['sub']=sub
            segdf=pd.concat([segdf,df],ignore_index=True) 
        elif sub in cf_subjects:
            if s2file == '/Users/bezdek/Library/CloudStorage/Box-Box/DCL_ARCHIVE/Documents/Events/exp152_fMRIneuralmechanisms/exp152_Session_2/data/e152037_exp152_session_2_2022_Mar_12_1202.csv':
                pass
            else:
                #s2file='/Users/bezdek/Library/CloudStorage/Box-Box/DCL_ARCHIVE/Documents/Events/exp152_fMRIneuralmechanisms/exp152_Session_2/data/e152037_exp152_session_2_2022_Mar_12_1229.csv'
                df=pd.read_csv(s2file)
                # select columns and rows:
                df=df[['movie','task','segment_2.rt','segment.rt']]
                df=df.rename(columns={'segment_2.rt':'segment_coarse.rt','segment.rt':'segment_fine.rt'})
                df['order']='coarse_fine'
                df = df[(df.task == 'segment') & (df.movie != 'legos2')]
                df['sub']=sub
                segdf=pd.concat([segdf,df],ignore_index=True) 
    except:
        print(s2file)

C:\Users\nguye\Box\DCL_ARCHIVE\Documents\Events\exp152_fMRIneuralmechanisms\exp152_Session_2\data\e152015_exp152_session_2_2021_Sep_27_1221.csv
C:\Users\nguye\Box\DCL_ARCHIVE\Documents\Events\exp152_fMRIneuralmechanisms\exp152_Session_2\data\e152019_exp152_session_2_2021_Nov_12_1104.csv
C:\Users\nguye\Box\DCL_ARCHIVE\Documents\Events\exp152_fMRIneuralmechanisms\exp152_Session_2\data\e152026_exp152_session_2_2021_Nov_22_1330.csv
C:\Users\nguye\Box\DCL_ARCHIVE\Documents\Events\exp152_fMRIneuralmechanisms\exp152_Session_2\data\e152027_exp152_session_2_2021_Dec_11_1337.csv
C:\Users\nguye\Box\DCL_ARCHIVE\Documents\Events\exp152_fMRIneuralmechanisms\exp152_Session_2\data\e152028_exp152_session_2_2021_Dec_17_0751.csv
C:\Users\nguye\Box\DCL_ARCHIVE\Documents\Events\exp152_fMRIneuralmechanisms\exp152_Session_2\data\e152029_exp152_session_2_2021_Dec_18_1018.csv
C:\Users\nguye\Box\DCL_ARCHIVE\Documents\Events\exp152_fMRIneuralmechanisms\exp152_Session_2\data\e152043_exp152_session_2_2022_Apr_11_1

In [33]:
# this segmentation.csv file was created by Matt Bezdek on 07/11/2022 for all subjects (the last subject were collected on 05/28/2022), use it.
segmentation_df = pd.read_csv("./segmentation.csv")
segmentation_df["sub"].unique()

array(['e152005', 'e152006', 'e152007', 'e152008', 'e152009', 'e152010',
       'e152011', 'e152013', 'e152014', 'e152016', 'e152017', 'e152018',
       'e152021', 'e152022', 'e152023', 'e152024', 'e152025', 'e152026',
       'e152028', 'e152030', 'e152031', 'e152032', 'e152033', 'e152034',
       'e152035', 'e152036', 'e152037', 'e152038', 'e152039', 'e152040',
       'e152041', 'e152042', 'e152044', 'e152045', 'e152046', 'e152047',
       'e152049', 'e152050', 'e152051', 'e152052', 'e152015', 'e152019',
       'e152020', 'e152027', 'e152029', 'e152003', 'e152004'],
      dtype=object)

In [31]:
segmentation_df.groupby('sub').count()

,movie,task,segment_coarse.rt,segment_fine.rt,order
sub,,,,,
e152003,4,4,4,0,4
e152004,4,4,4,0,4
e152005,8,8,4,4,8
e152006,8,8,4,4,8
e152007,8,8,4,4,8
e152008,8,8,4,4,8
e152009,8,8,4,4,8
e152010,8,8,4,4,8
e152011,8,8,4,4,8


In [68]:
# adapted from analyze_session_two.py
segmentation_df_long=pd.DataFrame()
i = 0
while i < len(segmentation_df):
    test=segmentation_df.iloc[i]
    if pd.notna(test['segment_fine.rt']):
        bds=ast.literal_eval(test['segment_fine.rt'])
        bds = [x for x in bds if x >= 0]
        testdf=pd.DataFrame(({'boundaries':bds}))
        testdf['grain']='fine'
        testdf['sub']=test['sub']
        testdf['order']=test['order']
        testdf['movie']=test['movie']   
        segmentation_df_long=pd.concat([segmentation_df_long,testdf],ignore_index=True)
    elif pd.notna(test['segment_coarse.rt']):
        bds = ast.literal_eval(test['segment_coarse.rt'])
        bds = [x for x in bds if x >= 0]
        testdf=pd.DataFrame(({'boundaries':bds}))
        testdf['grain']='coarse'
        testdf['sub']=test['sub']
        testdf['order']=test['order']
        testdf['movie']=test['movie'] 
        segmentation_df_long=pd.concat([segmentation_df_long,testdf],ignore_index=True)
    i+=1
segmentation_df_long

,boundaries,grain,sub,order,movie
0,8.701339,fine,e152005,fine_coarse,1.2.3
1,56.693312,fine,e152005,fine_coarse,1.2.3
2,80.317316,fine,e152005,fine_coarse,1.2.3
3,94.925336,fine,e152005,fine_coarse,1.2.3
4,143.277353,fine,e152005,fine_coarse,1.2.3
...,...,...,...,...,...
2845,240.356923,coarse,e152004,coarse,2.4.1
2846,314.900950,coarse,e152004,coarse,2.4.1
2847,401.164970,coarse,e152004,coarse,2.4.1
2848,584.540962,coarse,e152004,coarse,2.4.1


In [69]:
counter_balance = pd.read_excel(r"C:\Users\nguye\Box\DCL_ARCHIVE\Documents\Events\exp152_fMRIneuralmechanisms\forms&instructions\e152_counterbalance.xls", sheet_name='OA Participants')
counter_balance

,VC#####,fmriPrep Number,Experimenter,Exclude from analyses?,Age,Wore Mask in Scanner?,Name,Date,Notes,Follow-up date,Experimenter.1,Follow-up Notes
0,e152001,NaN,MB & GC,Y,NaN,Y,NaN,2020-09-19,Pilot scan. Audio wasn't working for first two...,NaT,NaN,NaN
1,e152002,NaN,MB & JZ,Y,NaN,Y,NaN,2020-09-10,Pilot scan. Final stimulus movies have not be...,NaT,NaN,NaN
2,e152003,sub-01,GC & MB & SM,N,NaN,Y,NaN,2021-05-10,No mirror until functional scans; re-ran struc...,2021-05-17,GC,Pressed the space bar multiple times to begin ...
3,e152004,sub-02,GC & MB & SM,N,NaN,Y,NaN,2021-05-12,No mirror until functional scans; re-ran struc...,2021-05-18,SM & GC,Participant accidentally skipped one of the fr...
4,e152005,NaN,SM,Y,NaN,NaN,NaN,2021-07-05,Behavior only,2021-07-13,SM,NaN
5,e152006,NaN,SM,Y,NaN,NaN,NaN,2021-07-12,Behavior only,2021-07-19,SM,NaN
6,e152007,sub-03,SM & MB & MS,N,NaN,Y,NaN,2021-07-17,Excellent participant; very still,2021-07-23,SM,coarse-fine
7,e152008,sub-04,SM & MB,N,NaN,Y,NaN,2021-07-24,Redid localizer scans due to metal in face mask,2021-07-31,SM,fine-coarse
8,e152009,sub-05,MB & MS,N,NaN,Y,NaN,2021-08-21,NaN,2021-08-28,SM,coarse-fine
9,e152010,sub-06,MB & MS,N,NaN,Y,NaN,2021-08-21,NaN,2021-08-29,MB,coarse-fine


In [70]:
# convert column VC##### and fmriPrepNumber to a dictionary
counter_balance_dict = counter_balance.set_index('VC#####').to_dict()['fmriPrep Number']
counter_balance_dict

{'e152001': nan,
 'e152002': nan,
 'e152003': 'sub-01',
 'e152004': 'sub-02',
 'e152005': nan,
 'e152006': nan,
 'e152007': 'sub-03',
 'e152008': 'sub-04',
 'e152009': 'sub-05',
 'e152010': 'sub-06',
 'e152011': 'sub-07',
 'e152012': nan,
 'e152013': 'sub-08',
 'e152014': 'sub-09',
 'e152015': 'sub-10',
 'e152016': 'sub-11',
 'e152017': 'sub-12',
 'e152018': 'sub-13',
 'e152019': 'sub-14',
 'e152020': 'sub-15',
 'e152021': 'sub-16',
 'e152022': 'sub-17',
 'e152023': 'sub-18',
 'e152024': 'sub-19',
 'e152025': 'sub-20',
 'e152026': 'sub-21',
 'e152027': 'sub-22',
 'e152028': 'sub-23',
 'e152029': 'sub-24',
 'e152030': 'sub-25',
 'e152031': 'sub-26',
 'e152032': 'sub-27',
 'e152033': 'sub-28',
 'e152034': 'sub-29',
 'e152035': 'sub-30',
 'e152036': 'sub-31',
 'e152037': 'sub-32',
 'e152038': 'sub-33',
 'e152039': 'sub-34',
 'e152040': 'sub-35',
 'e152041': 'sub-36',
 'e152042': 'sub-37',
 'e152043': 'sub-38',
 'e152044': 'sub-39',
 'e152045': 'sub-40',
 'e152046': 'sub-41',
 'e152047': '